In [0]:
https://docs.databricks.com/aws/en/views/dynamic

In [0]:
select * from lakehouse.naval_silver.sales_customers

In [0]:
-- Alias the field 'email' to itself (as 'email') to prevent the
-- permission logic from showing up directly in the column name results.
CREATE or replace VIEW lakehouse.naval_gold.sales_redacted AS
SELECT
  customerID,
  first_name,
  last_name,
  CASE WHEN
    is_account_group_member('user14@xsr890gmail.onmicrosoft.com') THEN 'REDACTED'
    ELSE email_address
  END AS email_address,
  state,
  country,
  gender
FROM lakehouse.naval_silver.sales_customers

In [0]:
-- Alias the field 'email' to itself (as 'email') to prevent the
-- permission logic from showing up directly in the column name results.
CREATE or replace VIEW lakehouse.naval_gold.sales_redacted AS
SELECT
  customerID,
  first_name,
  last_name,
  CASE WHEN
    is_account_group_member('account users') THEN email_address
    ELSE 'REDACTED'
  END AS email_address,
  state,
  country,
  gender
FROM lakehouse.naval_silver.sales_customers

In [0]:
%sql
CREATE OR REPLACE FUNCTION lakehouse.naval_gold.mask_email(email STRING)
RETURNS STRING
LANGUAGE PYTHON
DETERMINISTIC
AS $$
if email is None:
    return None
parts = email.split('@', 1)
if len(parts) != 2:
    return email
username, domain = parts
if len(username) <= 2:
    masked_username = '*' * len(username)
else:
    masked_username = '*' * (len(username) - 2) + username[-2:]
return f"{masked_username}@{domain}"
$$

In [0]:
-- Alias the field 'email' to itself (as 'email') to prevent the
-- permission logic from showing up directly in the column name results.
CREATE or replace VIEW lakehouse.naval_gold.sales_redacted AS
SELECT
  customerID,
  first_name,
  last_name,
  CASE WHEN
    is_account_group_member('account users') THEN lakehouse.naval_gold.mask_email(email_address)
    ELSE email_address
  END AS email_address,
  state,
  country,
  gender
FROM lakehouse.naval_silver.sales_customers

In [0]:
select lakehouse.naval_gold.mask_email("naval@gmail.com") as email_mask

In [0]:
grant use schema on schema lakehouse.naval_gold to `account users`;
grant select on table lakehouse.naval_gold.sales_redacted to `account users`

In [0]:
select * from lakehouse.naval_gold.sales_redacted